In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import os 
from info import *

while True:
    try:
        subject = input("PLease give me a subject name as the folder")
        path_loc = os.path.join(os.getcwd(), subject)
        os.mkdir(path_loc)
        break
    except:
        print(f"{subject} is not a valid folder name, please modify")
        continue

# Key to download pdf by default
options = webdriver.ChromeOptions()
chrome_prefs = {
    "download.prompt_for_download": False,
    "plugins.always_open_pdf_externally": True,
    "download.open_pdf_in_system_reader": False,
    "profile.default_content_settings.popups": 0,
    "download.default_directory": path_loc
}
options.add_experimental_option("prefs", chrome_prefs)

# pip install webdriver-manager
driver = webdriver.Chrome(options=options)


base_url = "https://bb.cuhk.edu.cn/"
coures_page = "webapps/portal/execute/tabs/tabAction?tab_tab_group_id=_2_1"
input_SID_xpath = r'''//*[@id="user_id"]'''
input_password_xpath = r'''//*[@id="password"]'''
login_button_xpath = r'''//*[@id="loginFormList"]/li[4]/input'''




driver.get(base_url)
from time import sleep
driver.find_element_by_xpath(input_SID_xpath).send_keys(S_ID)
driver.find_element_by_xpath(input_password_xpath).send_keys(password)


driver.find_element_by_xpath(login_button_xpath).send_keys(Keys.ENTER)
driver.get(base_url+coures_page)


# assert "Python" in driver.title
# elem = driver.find_element(By.NAME, "q")
# elem.send_keys("pycon")
# elem.send_keys(Keys.RETURN)
# assert "No results found." not in driver.page_source
# elem.clear()

Manually go to a main content page of a course

In [ ]:
IS_TEST = False # This only works for CSC3100 2022-2023 Fall Term course
# if not please manually go to the Main content page
# An example url for main content page is below
# https://bb.cuhk.edu.cn/webapps/blackboard/content/listContent.jsp?course_id=_8275_1&content_id=_274527_1&mode=reset
content_url = 'webapps/blackboard/content/listContent.jsp?'
course_page = "webapps/blackboard/execute/modulepage/view?course_id={course}&cmp_tab_id={cmp_tab}"
course_content_page = "webapps/blackboard/content/listContent.jsp?course_id={course}&content_id={content}"

folder_entry_xpath = '''//*[@id="containerdiv"]'''

if IS_TEST:
    course_id = "_8275_1" # Not useful, it is course specific one
    cmp_tab_id = '_17861_1'
    main_content_id = '_274527_1'
    # This may needs automation
    my_url = base_url + course_content_page.format(course=course_id, content=main_content_id)
    driver.get(my_url)
else:
    input("Ready? then Enter")

def download_files_under_this(total_file_links = [], pass_folder_links = [], depth = 0, max_depth = -1):
    if max_depth >= 0 and depth > max_depth:
        return
    # locate file links in this page
    try:
        content_list = driver.find_element(By.CLASS_NAME, "contentList")
        page_urls = content_list.find_elements(By.TAG_NAME, 'a')

        # print(len(set(page_urls)) == len(page_urls)) # whether all urls in this page are unique, seems always True
        
        for url in page_urls:
            page_url = url.get_attribute('href')
            if 'bbcswebdav' in page_url and page_url not in total_file_links:
                total_file_links.append(page_url)
                print("find new pdf link: ", page_url)
    except NoSuchElementException:
        # print("Content list not found in {driver.current_url}")
        pass
    except Exception as e:
        print(e)
    pass_folder_links.append(driver.current_url)
    # locate each folder
    folder_links = []
    try: # avoid stale references errors?
        folders = driver.find_element(By.XPATH, folder_entry_xpath).find_elements(By.CLASS_NAME, "read")
        for ele in folders:
            try:
                ref_link = ele.find_element(By.TAG_NAME, 'a').get_attribute('href')
                if content_url in ref_link and ref_link not in folder_links and ref_link not in pass_folder_links:
                    folder_links.append(ref_link)
                    print("Add folder link", ref_link)
                else:
                    pass
                    # print(f"{ref_link} is not a folder link in bb")
            except NoSuchElementException:
                # print(f"No element found in {ele.text}")
                continue
    except:
        pass
    # go through each folder
    for url in folder_links:
        driver.get(url)
        pass_folder_links.append(url)
        print("enter folder ", url)
        download_files_under_this(total_file_links, pass_folder_links, depth+1)
    
    if depth == 0:
        with open('file_links.txt', 'w') as f:
            for url in total_file_links:
                f.write(url + "\n")
            
        # download all file in file_links
        print("-"*10, "download starts", "-"*10)
        for idx, url in enumerate(total_file_links):
            if idx % 5 == 0:
                sleep(5) # let the network slow down
            driver.get(url)
        


total_file_links = []
pass_folder_links = []
depth = 0
download_files_under_this(total_file_links=total_file_links, pass_folder_links=pass_folder_links, depth=depth)
# All should be done at here, and if you think your download has finished after a few minutes then run next line

In [ ]:
driver.close() # Everything done!

In [ ]:
# Below are useless only for my testing

In [ ]:
content_ele = driver.find_element_by_xpath(folder_entry_xpath)
print("--"*5 + "possible folder names "+ "--"*5)
print(content_ele.text)

# Obtain each folder's reference link
# folders = driver.find_elements(By.CLASS_NAME, "read")
folders = content_ele.find_elements(By.CLASS_NAME, "read")

folder_links = []
ref_link = ""

for element in folders:
    try:
        a_tag = element.find_element(By.TAG_NAME, 'a')
    except NoSuchElementException:
        print(f"No element found in {element.text}")
        continue
    ref_link = a_tag.get_attribute('href')
    if content_url in ref_link:
        folder_links.append(ref_link)
    else:
        print(f"{ref_link} is not a folder link in bb")
print("--"*5 + "Folder links" + "--"*5)
print(folder_links)


In [ ]:
# Obtain download urls from current page 
file_links = []
try:
    content_list = driver.find_element(By.CLASS_NAME, "contentList")
    page_urls = content_list.find_elements(By.TAG_NAME, 'a')

    print(len(set(page_urls)) == len(page_urls))
    with open("file_link.txt", 'w') as f:
        with open('other_link.txt', 'w') as f2:
            for url in page_urls:
                page_url = url.get_attribute('href')
                if 'bbcswebdav' in page_url and page_url not in file_links:
                    file_links.append(page_url)
                    f.write(page_url + '\n')
                else:
                    f2.write(page_url + '\n')
except NoSuchElementException:
    print("Content list not found in {driver.current_url}")


# for url in page_urls:
#     print(url.get_attribute('href'))


In [ ]:
real_file_links = []

with open('pdf_link.txt', 'w') as f:
    
    for url in file_links:
        driver.get(url)
        real_file_links.append(driver.current_url)
        print(driver.current_url)
        f.write(driver.current_url + '\n')


In [ ]:
# Download pdf from url to specific folders https://scripteverything.com/download-pdf-selenium-python/
js_script = f''' 
			body = document.querySelector('body');
			element = document.createElement('a');
            element.href = "{real_file_links[5]}"
            element.download = "";
			text = document.createTextNode('Hello WOrld YABINYABIN');
			element.appendChild(text);
			body.append(element);
            //element.click()
'''

driver.execute_script(js_script)

In [ ]:
options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
"download.default_directory": "C:/Users/XXXX/Desktop", #Change default directory for downloads
"download.prompt_for_download": False, #To auto download the file
"download.directory_upgrade": True,
"plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
})
driver.create_options()

In [ ]:

driver.get('https://bb.cuhk.edu.cn/bbcswebdav/pid-317351-dt-content-rid-4972414_1/xid-4972414_1')

In [ ]:
for url in folder_links:
    driver.get(url)
    sleep(20)

In [ ]:
driver.close()